In [16]:
!pip install mstarpy numpy-financial

import json
import datetime
from collections import defaultdict
from mstarpy import Funds
import numpy_financial as npf


from google.colab import files
uploaded = files.upload()

file_name = list(uploaded.keys())[0]
with open(file_name, 'r') as file:
    data = json.load(file)


transactions = data['data'][0]['dtTransaction']


portfolio = defaultdict(lambda: defaultdict(list))
cash_flows = []


for txn in transactions:
    folio = txn['folio']
    scheme = txn['isin']
    txn_date = txn['trxnDate']
    txn_amount = float(txn['trxnAmount'])
    txn_units = float(txn['trxnUnits'])
    purchase_price = float(txn['purchasePrice'])


    portfolio[folio][scheme].append({
        'date': txn_date,
        'units': txn_units,
        'price': purchase_price
    })


    txn_date_obj = datetime.datetime.strptime(txn_date, "%d-%b-%Y")
    cash_flows.append((txn_date_obj, -txn_amount))


def get_latest_nav(isin):
    fund = Funds(term=isin, country="in")
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=365)
    history = fund.nav(start_date=start_date, end_date=end_date, frequency="daily")
    latest_nav = history[-1]['nav']
    return latest_nav


total_portfolio_value = 0
total_portfolio_gain = 0

for folio, schemes in portfolio.items():
    for scheme, transactions in schemes.items():
        latest_nav = get_latest_nav(scheme)
        remaining_units = 0
        acquisition_cost = 0

        for txn in transactions:
            units = txn['units']
            price = txn['price']
            remaining_units += units
            acquisition_cost += units * price

        current_value = remaining_units * latest_nav
        gain = current_value - acquisition_cost

        total_portfolio_value += current_value
        total_portfolio_gain += gain
today = datetime.datetime.now()
cash_flows.append((today, total_portfolio_value))

print(f"Total Portfolio Value: {total_portfolio_value}")
print(f"Total Portfolio Gain: {total_portfolio_gain}")

def calculate_xirr(cash_flows):

    dates, amounts = zip(*cash_flows)
    start_date = dates[0]
    times = [(date - start_date).days / 365.0 for date in dates]

    try:
        result = npf.irr(amounts)
        return result
    except Exception as e:
        print("Error in IRR calculation:", e)
        print("Cash flows (amounts):", amounts)
        print("Dates:", dates)
        return None

portfolio_irr = calculate_xirr(cash_flows)

if portfolio_irr is not None:
    print(f"Portfolio XIRR Approximation (IRR): {portfolio_irr * 100:.2f}%")
else:
    print(f"IRR calculation failed. Final portfolio value: {total_portfolio_value}")


Saving transaction_detail.json to transaction_detail (7).json
Total Portfolio Value: 4569440.0570003
Total Portfolio Gain: 1193223.7361168
Portfolio XIRR Approximation (IRR): 0.15%
